In [57]:
import matplotlib.pyplot as plt
import os
import re
import string
import tensorflow as tf
import pandas as pd

from utils import predicted_test_data_to_result_csv
from keras import layers, losses, Input, Model
from keras.layers import Dense, Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, BatchNormalization, Activation, Flatten, LSTM, SpatialDropout1D, Bidirectional
from keras.losses import sparse_categorical_crossentropy
from keras.metrics import sparse_categorical_accuracy
from keras.optimizers import Adam, SGD

In [58]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [59]:
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.10.1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [60]:
train_path = "data/base/goodreads_train.csv"
result_path = "data/base/goodreads_test.csv"
frac_ratio = 0.2

In [61]:
max_features = 1000  # Maximum vocab size.
sequence_length = 200

In [62]:
df = pd.read_csv(train_path, sep=",")

In [63]:
index = df[(df['rating'] == 0)].index
df.drop(index, inplace=True)
df.reset_index(inplace=True, drop=True)

In [64]:
x_train = df.sample(frac=frac_ratio)
x_val = df.drop(x_train.index)

In [65]:
y_train = x_train.pop('rating')
y_train = y_train - 1

y_val = x_val.pop('rating')
y_val = y_val - 1

In [66]:
x_train = x_train["review_text"]
x_val = x_val["review_text"]

In [67]:
raw_train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10, reshuffle_each_iteration=False)
raw_val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val)).shuffle(10, reshuffle_each_iteration=False)

In [68]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_spoilers = tf.strings.regex_replace(lowercase, '\*\* spoiler alert \*\*', ' ')
    return tf.strings.regex_replace(stripped_spoilers,
                                    '[%s]' % re.escape(string.punctuation),
                                    '')

In [69]:
vectorize_layer = tf.keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [70]:
epochs = 30
model_nb = 3

embedding_dim = 100
learning_rate = 0.01
batch_size = 3200
dropout_rate = 0.2
hidden_layer = 0

In [71]:
raw_train_dataset = raw_train_dataset.batch(batch_size=batch_size)
raw_val_dataset = raw_val_dataset.batch(batch_size=batch_size)

In [72]:
# Make a text-only dataset (without labels), then call adapt
train_text = raw_train_dataset.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

val_text = raw_val_dataset.map(lambda x, y: x)
vectorize_layer.adapt(val_text)

In [73]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [74]:
train_ds = raw_train_dataset.map(vectorize_text)
val_ds = raw_val_dataset.map(vectorize_text)

In [75]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [76]:
input_text = Input(shape=(sequence_length,))

embedding_layer = Embedding(max_features + 1, embedding_dim, input_length=sequence_length)(input_text)

spatial_dropout = SpatialDropout1D(dropout_rate)(embedding_layer)
# x = embedding_layer
# for i in range(hidden_layer):
#     x = SpatialDropout1D(dropout_rate)(x)
#     x = Bidirectional(LSTM(embedding_dim, dropout=dropout_rate, recurrent_dropout=dropout_rate, return_sequences=True))(x)
#
# lstm = Bidirectional(LSTM(25, return_sequences=False))(x)

lstm = LSTM(embedding_dim, dropout=dropout_rate, recurrent_dropout=dropout_rate)(spatial_dropout)

# global_max_pooling = GlobalMaxPooling1D()(lstm)
#
# dense = Dense(512, activation='relu')(lstm)

output = Dense(5, activation='softmax')(lstm)

rnn_model = Model(input_text, output)

rnn_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 200)]             0         
                                                                 
 embedding_2 (Embedding)     (None, 200, 100)          100100    
                                                                 
 spatial_dropout1d_2 (Spatia  (None, 200, 100)         0         
 lDropout1D)                                                     
                                                                 
 lstm_2 (LSTM)               (None, 100)               80400     
                                                                 
 dense_2 (Dense)             (None, 5)                 505       
                                                                 
Total params: 181,005
Trainable params: 181,005
Non-trainable params: 0
_____________________________________________________

In [77]:
rnn_model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate=learning_rate),
                  metrics=sparse_categorical_accuracy)

In [78]:
exp_name = f'rnn_model_{model_nb}_hidden_layers_{hidden_layer}_lr_{learning_rate}_bs_{batch_size}_dr_{dropout_rate}'

In [79]:
rnn_model.fit(train_ds,
              validation_data=val_ds,
              epochs=epochs,
              callbacks=[tf.keras.callbacks.TensorBoard("logs/rnn/" + exp_name)])

Epoch 1/30
55/55 [==============================] - 108s 2s/step - loss: 1.3949 - sparse_categorical_accuracy: 0.3558 - val_loss: 1.3791 - val_sparse_categorical_accuracy: 0.3623
Epoch 2/30
55/55 [==============================] - 108s 2s/step - loss: 1.3710 - sparse_categorical_accuracy: 0.3639 - val_loss: 1.3840 - val_sparse_categorical_accuracy: 0.3616
Epoch 3/30
55/55 [==============================] - 104s 2s/step - loss: 1.3789 - sparse_categorical_accuracy: 0.3603 - val_loss: 1.3833 - val_sparse_categorical_accuracy: 0.3604
Epoch 4/30
55/55 [==============================] - 106s 2s/step - loss: 1.3522 - sparse_categorical_accuracy: 0.3788 - val_loss: 1.2908 - val_sparse_categorical_accuracy: 0.4218
Epoch 5/30
55/55 [==============================] - 107s 2s/step - loss: 1.2457 - sparse_categorical_accuracy: 0.4526 - val_loss: 1.2008 - val_sparse_categorical_accuracy: 0.4733
Epoch 6/30
55/55 [==============================] - 108s 2s/step - loss: 1.1657 - sparse_categorical_accu

In [80]:
df_test = pd.read_csv(result_path, sep=",")

df_test_modified = df_test.drop(columns=[
    'user_id',
    'book_id',
    'review_id',
    'date_added',
    'date_updated',
    'read_at',
    'started_at',
    'n_votes',
    'n_comments'
], inplace=False)

In [81]:
export_model = tf.keras.Sequential([
    vectorize_layer,
    rnn_model
])

In [82]:
predicted_test_data = export_model.predict(df_test_modified)

14939/14939 [==============================] - 2565s 172ms/step


In [83]:
df_test.head()

,user_id,book_id,review_id,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,b9450d1c1f97f891c392b1105959b56e,7092507,5c4df7e70e9b438c761f07a4620ccb7c,** spoiler alert ** \n This is definitely one ...,Sat Nov 10 06:06:13 -0800 2012,Sun Nov 11 05:38:36 -0800 2012,Sun Nov 11 05:38:36 -0800 2012,Sat Nov 10 00:00:00 -0800 2012,1,0
1,b9450d1c1f97f891c392b1105959b56e,5576654,8eaeaf13213eeb16ad879a2a2591bbe5,"** spoiler alert ** \n ""You are what you drink...",Fri Nov 09 21:55:16 -0800 2012,Sat Nov 10 05:41:49 -0800 2012,Sat Nov 10 05:41:49 -0800 2012,Fri Nov 09 00:00:00 -0800 2012,1,0
2,b9450d1c1f97f891c392b1105959b56e,15754052,dce649b733c153ba5363a0413cac988f,Roar is one of my favorite characters in Under...,Fri Nov 09 00:25:50 -0800 2012,Sat Nov 10 06:14:10 -0800 2012,Sat Nov 10 06:14:10 -0800 2012,Fri Nov 09 00:00:00 -0800 2012,0,0
3,b9450d1c1f97f891c392b1105959b56e,17020,8a46df0bb997269d6834f9437a4b0a77,** spoiler alert ** \n If you feel like travel...,Thu Nov 01 00:28:39 -0700 2012,Sat Nov 03 11:35:22 -0700 2012,Sat Nov 03 11:35:22 -0700 2012,Thu Nov 01 00:00:00 -0700 2012,0,0
4,b9450d1c1f97f891c392b1105959b56e,12551082,d11d3091e22f1cf3cb865598de197599,3.5 stars \n I read and enjoyed the first two ...,Thu Oct 18 00:57:00 -0700 2012,Mon Apr 01 23:00:51 -0700 2013,Sat Mar 30 00:00:00 -0700 2013,Fri Mar 29 00:00:00 -0700 2013,0,0


In [84]:
predicted_test_data_to_result_csv(df_test, predicted_test_data, exp_name)